In [3]:
import os
import sys
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define and set the working directory
new_directory = '/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/00 - Pulizia dati'
if os.path.isdir(new_directory):
    os.chdir(new_directory)
    sys.path.append(new_directory)
    print("Current directory:", os.getcwd())
else:
    print("Directory does not exist:", new_directory)

Mounted at /content/drive
Current directory: /content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/00 - Pulizia dati


# Let's load out datasets

In [4]:
import pandas as pd
import os


# Load the Excel files
veicoli = pd.read_excel("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/00 - Pulizia dati/veicoli.xlsx")
statocivile = pd.read_excel("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/00 - Pulizia dati/statocivile.xlsx")
altimetria = pd.read_excel("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/00 - Pulizia dati/altimetrie.xlsx")

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


# VEICOLI
## Let's clean veicoli datatset

In [5]:
# Remove rows 1, 2, 3, and 5 (in Python: indices 0, 1, 2, 4)
veicoli = veicoli.drop(index=[0, 1, 2, 3, 5])

# Set the first remaining row as the new header
veicoli.columns = veicoli.iloc[0]
veicoli = veicoli.drop(veicoli.index[0])

# Rename key columns for clarity
veicoli = veicoli.rename(columns={veicoli.columns[0]: "Comune", veicoli.columns[16]: "Tot_veicoli", 'Autovetture  ':"Autovetture"})

# Convert the 'Autovetture' column to numeric
veicoli["Autovetture"] = pd.to_numeric(veicoli["Autovetture"], errors='coerce')

# Calculate the number of electric vehicles (11.5 per 1000 cars)
veicoli["Auto_elettriche"] = (veicoli["Autovetture"] * 11.5 / 1000).round(0)

# STATO CIVILE

In [6]:
# Remove rows 1, 2, 3, and 5 (in Python: indices 0, 1, 2, 4)
statocivile = statocivile.drop(index=[0, 1, 2, 3, 4, 5,7])

# Set the first remaining row as the new header
statocivile.columns = statocivile.iloc[0]
statocivile = statocivile.drop(statocivile.index[0])

# Rinominare la prima colonna in 'Comune' per maggiore chiarezza
statocivile = statocivile.rename(columns={statocivile.columns[0]: "Comune"})

# Rimuovere le colonne specificate (se esistono, altrimenti ignora)
columns_to_drop = [
    "Unito/a civilmente",
    "Già in unione civile (per decesso del partner)",
    "Già in unione civile (per scioglimento unione)"
]
statocivile = statocivile.drop(columns=[col for col in columns_to_drop if col in statocivile.columns])

# Rinominare la colonna numero 6 (indice 5) in 'Pop_tot'
if len(statocivile.columns) > 5:
    statocivile = statocivile.rename(columns={statocivile.columns[5]: "Pop_tot"})

# Visualizzare le prime righe e i nomi delle colonne
print(statocivile.head())
print(statocivile.columns)

6          Comune Nubile/celibe   Coniugata/o   Divorziata/o   Vedova/o    \
8   Acquafredda               639           720             60        106   
9          Adro              3092          3300            228        531   
10     Agnosine               671           793             60        130   
11   Alfianello               996          1112             47        176   
12         Anfo               179           204             25         39   

6  Pop_tot Già in unione civile (per decesso del partner)    \
8       ..                                               ..   
9       ..                                               ..   
10      ..                                               ..   
11      ..                                               ..   
12      ..                                               ..   

6  Già in unione civile (per scioglimento unione)   Totale    
8                                                ..     1525  
9                               

# ALTIMETRIA

In [7]:
altimetria.head()

,COD_REG,COD_PRO,PRO_COM,NOME,SUPERFICIE (in KMQ),ALT_MIN,ALT_MAX,RANGE,MEDIA,MEDIANA,STD
0,12,58,58120,Fiumicino,213.8944,-3,164,167,31.688450,5,39.686070
1,5,29,29001,Adria,113.3900,-3,8,11,1.057863,1,1.244239
2,5,29,29030,Loreo,39.8442,-3,7,10,0.464185,0,0.770743
3,5,29,29052,Porto Viro,133.7686,-3,6,9,0.555705,1,0.553657
4,8,38,38017,Ostellato,173.3434,-3,8,11,1.784421,1,0.977924


In [8]:
# Rinomina le colonne
altimetria.columns = [
    "COD_REG", "COD_PRO", "PRO_COM", "Comune", "Superficie",
    "Alt_min", "Alt_max", "Range_alt", "Alt_media", "Alt_mediana", "Alt_std"
]

# Mantieni solo le righe per i comuni della provincia di Brescia (COD_PRO == 17)
altimetria = altimetria[altimetria["COD_PRO"] == 17].copy()

# Correggi i nomi dei comuni specifici PRIMA di rimuovere le colonne
altimetria["Comune"] = altimetria["Comune"].replace({
    "Tremosine": "Tremosine sul Garda",
    "Puegnago sul Garda": "Puegnago del Garda"
})

# Ora puoi rimuovere le prime 3 colonne
altimetria = altimetria.drop(columns=["COD_REG", "COD_PRO", "PRO_COM"])

# Verifica
print(altimetria.head())
print(altimetria.columns)
print(altimetria.shape)

         Comune  Superficie  Alt_min  Alt_max  Range_alt  Alt_media  \
1595       Salò     27.3056       29      910        881  188.87490   
1745  Pralboino     17.1625       35       53         18   47.66404   
1756     Fiesse     16.0162       35       47         12   42.50407   
1779     Seniga     13.5721       36       52         16   44.07557   
1796    Milzano      8.4890       37       55         18   50.22227   

      Alt_mediana     Alt_std  
1595          117  160.659300  
1745           48    2.846034  
1756           43    2.193938  
1779           45    4.439691  
1796           51    3.191340  
Index(['Comune', 'Superficie', 'Alt_min', 'Alt_max', 'Range_alt', 'Alt_media',
       'Alt_mediana', 'Alt_std'],
      dtype='object')
(206, 8)


# Merge the datasets

In [9]:
# Merge veicoli e statocivile sul campo 'Comune'
df_merged = pd.merge(veicoli, statocivile, on="Comune", how="inner")

# Pulisci gli spazi finali (e iniziali) nei nomi dei comuni di df_merged
df_merged["Comune"] = df_merged["Comune"].str.strip()

# Merge del risultato con altimetria
df_merged = pd.merge(df_merged, altimetria, on="Comune", how="inner")

# Visualizza il numero di righe e colonne del dataset finale
print(df_merged.shape)
print(df_merged.head())

(205, 33)
        Comune  Autovetture Autobus e filobus   Autocarri   Motrici    \
0  Acquafredda          996                   0         185         0   
1         Adro         4812                   1         699        19   
2     Agnosine         1242                   0         228         5   
3   Alfianello         1585                   0         221         7   
4         Anfo          307                   0          41         0   

  Rimorchi   Motocicli   Motocarri   Altri veicoli    \
0          7         156           8               0   
1         54         797          36               0   
2          8         264          17               0   
3         16         218           7               0   
4          0          59           9               0   

  Autovetture circolanti Euro 0    ...  \
0                              47  ...   
1                             237  ...   
2                             104  ...   
3                             115  ...   
4   

# Save it

In [10]:
# Save the merged dataframe to an Excel file
df_merged.to_excel("dataset_unito.xlsx", index=False)

print("File 'dataset_unito.xlsx' has been saved successfully.")

File 'dataset_unito.xlsx' has been saved successfully.


In [11]:
total_electric_cars = df_merged["Auto_elettriche"].sum()
print(f"Total number of electric cars: {total_electric_cars}")

Total number of electric cars: 9661.0
